In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [8]:
import requests
years=range(2010,2017)
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_"
responses = [requests.get(url+str(year)) for year in years ]
print responses[0].url
print responses[0].text[:1000] + "..."
print len(responses)

https://en.wikipedia.org/wiki/List_of_American_films_of_2010
<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of American films of 2010 - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_American_films_of_2010","wgTitle":"List of American films of 2010","wgCurRevisionId":755839865,"wgRevisionId":755839865,"wgArticleId":25636852,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Incomplete film lists","Commons category with local link same as on Wikidata","2010 in American cinema","Lists of 2010 films by country or language","Lists of American films by year"],"wgBreakFrames":false,"wgPageC

In [12]:
from bs4 import BeautifulSoup
soups = [BeautifulSoup(response.text) for response in responses]
print soups[6].head()

[<meta charset="utf-8"/>, <title>List of American films of 2016 - Wikipedia</title>, <script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>, <script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_American_films_of_2016","wgTitle":"List of American films of 2016","wgCurRevisionId":760866101,"wgRevisionId":760866101,"wgArticleId":45452962,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category with local link same as on Wikidata","Lists of American films by year","Lists of 2016 films by country or language","2016 in American cinema"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

In [37]:
soups[2].select('table.wikitable')[0].select('th')[0].text

u'Title'

In [75]:
tables = [ soup.select('table.wikitable') for soup in soups]

In [81]:
len(tables)

7

In [82]:
columns = [table[0].select('th') for table in tables]

In [83]:
features = zip(years,[[c.text for c in col] for col in columns])

In [84]:
features

[(2010,
  [u'Title',
   u'Director',
   u'Cast(Subject of documentary)',
   u'Genre',
   u'Notes']),
 (2011,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes']),
 (2012,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes',
   u'Release date']),
 (2013,
  [u'Title',
   u'Director',
   u'Cast (Subject of documentary)',
   u'Genre',
   u'Notes',
   u'Release date',
   u'Budget/BoxOffice $million']),
 (2014,
  [u'Opening', u'Title', u'Director', u'Cast', u'Genre', u'Notes', u'Ref.']),
 (2015,
  [u'Opening',
   u'Title',
   u'Director',
   u'Cast',
   u'Genre',
   u'Notes',
   u'Ref.',
   u'J\nA\nN\nU\nA\nR\nY',
   u'2',
   u'9',
   u'27']),
 (2016,
  [u'Opening', u'Title', u'Director', u'Cast', u'Genre', u'Notes', u'Ref.'])]

In [109]:
vintages = [[t.select('td') for t in table ]for table in tables]

In [148]:
data={'2010':[] , '2011':[], '2012':[], '2013':[], '2014':[], '2015':[], '2016':[]}
for year,vintage in zip(years,vintages):
    for releases in vintage:
        credits =[release.text for release in releases]
        data[str(year)].append(credits)
    

In [149]:
data['2010'][0]

[u'127 Hours',
 u'Danny Boyle',
 u'James Franco, Amber Tamblyn, Kate Mara, Cl\xe9mence Po\xe9sy, Lizzy Caplan',
 u'Biography, Drama',
 u'Fox Searchlight Pictures\nAdapted from Between a Rock and a Hard Place by Aron Ralston',
 u'8: The Mormon Proposition',
 u'Reed Cowan',
 u"The Church of Jesus Christ of Latter-day Saints' involvement in the 2008 California Proposition 8. Narrated by Dustin Lance Black.",
 u'Documentary',
 u'Red Flag Releasing',
 u'The A-Team',
 u'Joe Carnahan',
 u'Liam Neeson, Bradley Cooper, Quinton "Rampage" Jackson, Sharlto Copley, Jessica Biel, Patrick Wilson, Brian Bloom',
 u'Action, Adventure',
 u'20th Century Fox\nBased on the television series of the same name.',
 u'A Little Help',
 u'Michael J. Weithorn',
 u"Jenna Fischer, Chris O'Donnell, Rob Benedict, Arden Myrin, Daniel Yelsky",
 u'Comedy',
 u'Secret Handshake Entertainment',
 u'Adventures of Power',
 u'Ari Gold',
 u'Ari Gold, Michael McKean, Jane Lynch, Shoshannah Stern, Chiu Chi Ling, Adrian Grenier',
 u

In [134]:
data['2016'][3].pop()

u'F. Javier Guti\xe9rrez'

In [150]:
movies = { '2010':[]}
for data in data['2010']:
    while data:
        try:
            notes = data.pop()
            genre = data.pop()
            cast= data.pop()
            director = data.pop()
            title= data.pop()
            movies['2010'].append({'title':title, 'director':director, 'cast':cast, 'genre': genre, 'notes': notes})
        except IndexError:
            continue

In [151]:
movies

{'2010': [{'cast': u'Michael Cera, Portia Doubleday, Jean Smart, Mary Kay Place, Justin Long, Ray Liotta, Steve Buscemi, Zack Galifianakis, Ari Graynor, Fred Willard',
   'director': u'Miguel Arteta',
   'genre': u'Comedy-drama',
   'notes': u'Dimension Films\nBased on the novel of the same name by C. D. Payne',
   'title': u'Youth in Revolt'},
  {'cast': u'Antonio Banderas, Josh Brolin, Anthony Hopkins, Gemma Jones, Freida Pinto, Lucy Punch, Naomi Watts',
   'director': u'Woody Allen',
   'genre': u'Dramedy',
   'notes': u'Sony Pictures Classics',
   'title': u'You Will Meet a Tall Dark Stranger'},
  {'cast': u'Kristen Bell, Jamie Lee Curtis, Sigourney Weaver, Odette Yustman, Kristin Chenoweth, Victor Garber, Betty White, Kyle Bornheimer, Christine Lakin, Cloris Leachman',
   'director': u'Andy Fickman',
   'genre': u'Comedy',
   'notes': u'Touchstone Pictures',
   'title': u'You Again'},
  {'cast': u'Dan Aykroyd, Justin Timberlake, Anna Faris, Tom Cavanagh, T. J. Miller',
   'directo